In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./Data/cat_clean.csv')

In [3]:
df.head()

,Age,Gender,Neuter_status,Breed_group,Weaning_age,Outdoors,Other_cats,Activity_level,Contact_people,Aggression_stranger,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming,Wool_sucking,Behaviour_problem
0,4.0274,2,1,BEN,8,0,1,4,5,1,1,1,2,1,1.0,0.0,1.0
1,2.1096,2,1,BEN,8,0,1,5,4,1,1,1,3,3,1.0,0.0,1.0
2,7.6822,1,1,BUR,4,0,1,4,5,1,1,1,2,1,4.0,3.0,2.0
3,5.0027,1,1,BUR,4,4,0,5,5,1,1,2,1,1,1.0,0.0,1.0
4,5.0137,1,1,EUR,4,5,1,4,5,1,1,1,2,1,1.0,0.0,1.0


In [ ]:
# BINARY DATA SET
df_bin = df.copy()
# 0-2 = no sucking, 3-7 = sucking 
df_bin['ws_binary'] = df_bin['Wool_sucking'].replace({1:0, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1})
df_bin.drop(columns='Wool_sucking', inplace=True)
df_bin.head()

df_bin[['ws_binary']].describe()

In [ ]:
# Assign BINARY X and y
X_bin = df_bin.drop(columns='ws_binary')
y_bin = df_bin['ws_binary']

**Notes:**  
* I'm not sure if I will scale the age column for my distance-based classifiers. I feel like doing so will lose an aspect of interpretability if the goal here is to be able to predict if a cat you're going to adopt would be a wool-sucker or not. However, I do understand the importance of magnitiude in algorithms of that nature. 
* I am going to reocde `Behaviour_problem` as a binary of yes or no (compared to I don't know, no, yes dx'd by vet, yes dx'd by self). 
* I wanted to do multiclass classification on wool_sucking to capture more of the variance, however I less than 100 samples for classes 6 and 7, so I 
* I will try logistic regression and random forest models. If neither of those perform well, I will look into a boosted model.

In [ ]:
# PCA data set
pca_complete_df['ws_binary'] = pca_complete_df['Wool_sucking'].replace({1:0, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1})
pca_complete_df.drop(columns='Wool_sucking', inplace=True)
pca_complete_df = pd.get_dummies(pca_complete_df, columns=['Breed_group'], prefix='B')
# assign X and y 
X_pca = pca_complete_df.drop(columns='ws_binary')
y_pca = pca_complete_df['ws_binary']
# train test split 
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y_pca, 
                                                                    test_size=0.3, random_state=42, stratify=y_pca)


log_PCA = LogisticRegression().fit(X_train_pca, y_train_pca)
y_pred_pca = log_PCA.predict(X_test_pca)

add_to_master('Def LogReg Binary + PCA', y_test_pca, y_pred_pca)

In [ ]:
# just drop behaviour_problem
X_nbp = X_bin.drop(columns='Behaviour_problem')
X_train_nbp, X_test_nbp, y_train_nbp, y_test_nbp = train_test_split(X_nbp, y_bin, 
                                                                    test_size=0.3, random_state=42, stratify=y_bin)
log_nbp = LogisticRegression().fit(X_train_nbp, y_train_nbp)
y_pred_nbp = log_nbp.predict(X_test_nbp)

add_to_master('Def LogReg Binary + drop behavior', y_test_nbp, y_pred_nbp)

In [ ]:
# BINARY (normal)- no scaling as logistic regression is not sensitive to magnitude 

# Instantiate and fit model
log = LogisticRegression().fit(X_train, y_train)

# Calculate Prediction, Predicted Probabilities, FPR, TPR
y_pred = log.predict(X_test)
y_pred_probs = log.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

In [ ]:
# Graph ROC with AUC
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr)
plt.text(0.5, 0.3, f'AUC = {roc_auc_score(y_test, y_pred_probs):.4f}', fontsize=12, ha='center')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Log Reg ROC Curve - Binary")
plt.show();

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=log.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=log.classes_)
disp.plot()
plt.show();

In [ ]:
# add_to_master(model_description, y_true, y_predicted)
add_to_master('Default Logistic Regression Binary', y_test, y_pred)

# check out master_scores
master_scores.sort_values('Recall', ascending=False).head()

### 1.1.1 Binary Logistic Regression Interpretations:
The model does really well with true negatives, and pretty stinking awful at true positives. This model performs okay in the broad scope but it truly does not live up to the intended goal of being able to accurately predict if a kitty I'm looking to adopt will wool-suck or not. Actually it pretty much just predicts that they all won't. I am going to make recall my main metric to watch out for for model comparison as that is where I assume my models will continue to struggle. 

In [ ]:
# arugula
pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_data)
pca_columns = [f"PC{i}" for i in range(1,3)]
pca_df = pd.DataFrame(pca_data, columns=pca_columns)
pca_complete_df = pd.concat([pca_df, df.drop('Behaviour_problem', axis=1)], axis=1)

In [ ]:
# BINARY DATASET WITH PCA 
pca_bin = pca_complete_df.copy()
pca_bin['ws_binary'] = pca_bin['Wool_sucking'].replace({1:0, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1})
pca_bin.drop(columns='Wool_sucking', inplace=True)
pca_bin = pd.get_dummies(pca_bin, columns=['Breed_group'], prefix='B')
# assign X and y 
X_pca_bin = pca_bin.drop(columns='ws_binary')
y_pca_bin = pca_bin['ws_binary']
# train test split 
X_train, X_test, y_train, y_test = train_test_split(X_pca_bin, y_pca_bin, 
                                                                    test_size=0.3, random_state=42, stratify=y_pca_bin)

In [ ]:
# MULTICLASS DATASET WITH PCA 
pca_m = pca_complete_df.copy()
pca_m = pd.get_dummies(pca_m, columns=['Breed_group'], prefix='B')
# assign X and y 
X_pca_m = pca_m.drop(columns='Wool_sucking')
y_pca_m = pca_m['Wool_sucking']
# train test split 
X_trains, X_tests, y_trains, y_tests = train_test_split(X_pca_m, y_pca_m, 
                                                                    test_size=0.3, random_state=42, stratify=y_pca_m)

## 1.4 Logistic Regression Parameter Tuning:
As expected, the binary performed the best. However, there was one common thread: all models essentially predicted each sample as the majority class. Let's quickly check out this class imbalance. There's no point in tuning a model if you're still feeding it garbage! 

In [ ]:
y_bin.value_counts(normalize=True)

Quite the imbalance! Let's employ some resampling techniques to see if we find any improvement. Some resampling techniques include random undersampling (RUS), random oversampling (ROS), and Synthetic Minority Oversampling Technique (SMOTE). I predict the ROS will perform best as our dataset is already so small; I believe reducing datapoints even more would not be helpful. Oversampling, however, can have a tendency to overfit as it introduces redundant information. 

In [ ]:
X_age_scaled = X_train.copy()
X_age_scaled_test = X_test.copy()

scaler_age = MinMaxScaler(feature_range=(0,5))
scaler_age.fit(X_age_scaled['Age'].values.reshape(-1,1))

X_age_scaled['Age'] = scaler_age.transform(X_age_scaled['Age'].values.reshape(-1, 1))
X_age_scaled_test['Age'] = scaler_age.transform(X_age_scaled_test['Age'].values.reshape(-1,1))

In [ ]:
# instantiate model
knn_scaled = KNeighborsClassifier()

# instantiate grid search
param_grid = {'n_neighbors':[3, 5, 7, 9]}
grid_search = GridSearchCV(knn_scaled, param_grid, cv=5, scoring='recall')

# Fit it
grid_search.fit(X_age_scaled, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)

# Predicted and predicted probabilities
best_scaled = grid_search.best_estimator_
y_pred = best_scaled.predict(X_age_scaled_test)
y_pred_probs = best_scaled.predict_proba(X_age_scaled_test)[:,1]

In [ ]:
# multiclass KNN 
X_age_scaled = X_trains.copy()
X_age_scaled_tests = X_tests.copy()

scaler_age = MinMaxScaler(feature_range=(0,5))
scaler_age.fit(X_age_scaled['Age'].values.reshape(-1,1))

X_age_scaled['Age'] = scaler_age.transform(X_age_scaled['Age'].values.reshape(-1, 1))
X_age_scaled_test['Age'] = scaler_age.transform(X_age_scaled_tests['Age'].values.reshape(-1,1))


ks = [3, 5, 7]

for k in ks:
    knn_m = KNeighborsClassifier(n_neighbors=k).fit(X_trains, y_trains)
    y_preds = knn_m.predict(X_tests)
    # add_to_master(model_description, y_true, y_predicted)
    add_to_master(f'Multiclass KNN k={k}', y_tests, y_preds, binary=False)

In [ ]:
BEST_model = RandomForestClassifier(class_weight='balanced', max_features='auto',
                                    min_samples_split=10, n_estimators=100, random_state=42)
BEST_model.fit(X_train, y_train)
y_pred = BEST_model.predict(X_test)

In [ ]:
# Recap:

Only the binary classifiers did a good job! We tried Logistic Regression, a Random Forest, and KNN.

We will be carrying on with the following model: 
`log_rus` (LogisticRegression with default parameters trained on randomly undersampled data.)  

It has the highest recall, which is important because initial modelling suggested that would be my dataset's weakness. 

In [ ]:
# andrea code snippet
# Import regression modules
#import statsmodels.api as sm
#from statsmodels.formula.api import ols
#import statsmodels.formula.api as smf

#m = smf.ols(formula = 'PRICE ~ AGE + TAX',data = bos).fit()
#print(m.summary())

In [ ]:
# ORDINAL
import statsmodels.api as sm
start_time = time.time()

# this algorithm prefers numpy arrays 
X_train_o = X_train.values
y_train_o = y_train.values

# Add constant to training
X_train_o = sm.add_constant(X_train_o)

# Instantiate model
ordinal_model = sm.MNLogit(y_train_o, X_train_o)

# Fit model
ordinal_results = ordinal_model.fit_regularized(method='l1', alpha=0.5)

end_time = time.time()
elapse_time = round((end_time - start_time), 2)
print(f"Elapsed Time {elapse_time} seconds.")

# Add constant to testing 
X_test_o = X_test.values
X_test_o = sm.add_constant(X_test_o)

# Calculate Predictions and Predicted Probabilities 
y_pred_probs = ordinal_results.predict(X_test_o)
y_preds = np.argmax(y_pred_probs, axis=1)  # Convert predicted probabilities to class labels

In [ ]:
# I got 1.0's across the board for ALL its performance metrics, which seems fishy. 
# I'm just going to make sure I have a fresh training and testing set just in case. 
X = df_comp_encoded.drop(columns='ws_compressed')
y = df_comp_encoded['ws_compressed']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


